In [1]:
import sys
sys.path.append('/Users/nora/Code/RMG-Py')

import cantera as ct
import re
import os 
from rmgpy.molecule import Molecule

In [2]:
#first, edit the file so it doesn't overwrite the smiles
path = './pfca_final.yaml'   #load in thermo file
with open(path, 'r') as f: 
    lines = f.readlines()

SMILES_indices = []
for ind, line in enumerate(lines): 
    if "SMILES" in line: 
        SMILES_indices.append(ind)

indices_to_remove = []
#now look at the next three lines after these SMILE lines
for ind in SMILES_indices: 
    for x in range(1,5):
        next_ind = ind + x
        if "note:" in lines[next_ind]: 
            #we will not be saving this line
            indices_to_remove.append(next_ind)


In [ ]:
edited_lines_to_write = [lines[ind] for ind,line in enumerate(lines) if ind not in indices_to_remove]
#now write the edited lines to a new file
with open('./pfca_final_EDITED.yaml', 'w') as f:
    f.writelines(edited_lines_to_write)


In [3]:
#great, now we edited the files, now let's read the file
path = './pfca_final_EDITED.yaml'
gas = ct.Solution(path)     

fluorinated_species = []

for species in gas.species():
    if "F" in species.composition.keys():
        fluorinated_species.append(species)



In [4]:
#which fluorinated species do we have the smiles for? 
for spc in fluorinated_species:
    if "SMILES" not in spc.thermo.input_data['note']:
        print(spc)
#good, looks like its in all of them! 

In [32]:
gas.species(170).thermo.cp(1000)


30169.854787768654

In [33]:
#this will be the final string that we will use to write to the thermo file
final_string = ''
#this will be the species dictionary with key = label, value = adjacency_list
species_structure_dictionary = dict()

for ind, spc in enumerate(fluorinated_species): 
    #get the label
    label = spc.name

    if "SMILES" in spc.thermo.input_data['note']: 
        #get the adjacency list 
        smiles_str = spc.thermo.input_data['note'][9:] #cutting off the 'SMILES = '
        adjacency_list = Molecule(smiles=smiles_str).to_adjacency_list() #make into an adjacency list string
        species_structure_dictionary[label] = adjacency_list #append to species structure dictionary

        #get thermos
        [low_temp, mid_temp, high_temp] = spc.thermo.input_data['temperature-ranges']
        assert high_temp == spc.thermo.max_temp, "somethings not matching up"
        assert low_temp == spc.thermo.min_temp, "somethings not matching up"
        [first_range_coeffs, second_range_coeffs] = spc.thermo.input_data['data']
        cp0 = spc.thermo.cp(0)/1000 #cantera has it in J/kmol/K and RMG wants it in J/mol/K

        #now add to the final string
        string_to_add = f'''

entry(
    index = {ind},
    label = "{label}",
    molecule = 
"""
{adjacency_list}""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs={first_range_coeffs}, Tmin=({low_temp},'K'), Tmax=({mid_temp},'K')),
            NASAPolynomial(coeffs={second_range_coeffs}, Tmin=({mid_temp},'K'), Tmax=({high_temp},'K')),
        ],
        Tmin = ({low_temp},'K'),
        Tmax = ({high_temp},'K'),
    ),
    reference = Reference(authors=["Caroline Rocchio", "C. Franklin Goldsmith"], title="pfca_final", year="2024"),
    referenceType = "Theory",
)
'''
        final_string+=string_to_add


In [34]:
print(final_string)



entry(
    index = 0,
    label = "HF",
    molecule = 
"""
1 F u0 p3 c0 {2,S}
2 H u0 p0 c0 {1,S}
""",
    thermo = NASA(
        polynomials = [
            NASAPolynomial(coeffs=[-962.551409, 24.83162092, 3.26819899, 0.001039587354, -2.260501733e-06, 2.279324001e-09, -7.21878178e-13, -33951.0025, 2.129465739], Tmin=(100.0,'K'), Tmax=(1000.0,'K')),
            NASAPolynomial(coeffs=[725508.525, -1485.62134, 3.85872564, 0.000710849781, -2.095425721e-07, 3.031118495e-11, -1.631732228e-15, -23482.29903, -3.22409796], Tmin=(1000.0,'K'), Tmax=(6000.0,'K')),
        ],
        Tmin = (100.0,'K'),
        Tmax = (6000.0,'K'),
    ),
    reference = Reference(authors=["Caroline Rocchio", "C. Franklin Goldsmith"], title="pfca_final", year="2024"),
,
    referenceType = "Theory",
)


entry(
    index = 1,
    label = "FC_O_OH",
    molecule = 
"""
1 O u0 p2 c0 {2,D}
2 C u0 p0 c0 {1,D} {3,S} {4,S}
3 O u0 p2 c0 {2,S} {5,S}
4 F u0 p3 c0 {2,S}
5 H u0 p0 c0 {3,S}
""",
    thermo = NASA(
        po

In [35]:
with open('PFCA_thermo.py', 'w') as f:
    f.write(final_string)